In [ ]:
import os
import sys

from google.colab import drive

drive.mount('/content/drive')

project_folder = '/content/drive/MyDrive/Filterify'
sys.path.append(project_folder)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# **Album Cover Embedding**

In [ ]:
album_folder = '/content/drive/MyDrive/Filterify/Archive/covers'

image, label = [], []

for root, dirs, files in os.walk(album_folder):
    for file in files:
        if file.endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(root, file)
            image.append(image_path)

            label.append(file)

from PIL import Image

for index, img in enumerate(image):
  img = Image.open(img).convert("RGB")
  processed_img = processor(images=img, return_tensors="pt")

  with torch.no_grad():
    image_features = model.get_image_features(**processed_img)

  image_features_np = image_features.numpy()
  np.save(f"/content/drive/MyDrive/Filterify/Archive/embedding_features/{label[index]}", image_features_np)

# **Text Embedding**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Filterify/Archive/dataset.csv')

for idx, row in df.iterrows():
    artists = row['artists']
    track_name = row['track_name']
    tone_from_desc = row['tone_from_des']
    theme_from_desc = row['theme_from_des']
    tone_from_album = row['tone']
    theme_from_album = row['theme']

    texts = [tone_from_desc, theme_from_desc, tone_from_album, theme_from_album]

    inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        text_embeddings = model.get_text_features(**inputs)

    try:
      np.save(f"/content/drive/MyDrive/Filterify/Archive/tone_from_desc/{artists}_{track_name}", text_embeddings[0].numpy())
      np.save(f"/content/drive/MyDrive/Filterify/Archive/theme_from_desc/{artists}_{track_name}", text_embeddings[1].numpy())
      np.save(f"/content/drive/MyDrive/Filterify/Archive/tone_from_cover/{artists}_{track_name}", text_embeddings[2].numpy())
      np.save(f"/content/drive/MyDrive/Filterify/Archive/theme_from_cover/{artists}_{track_name}", text_embeddings[3].numpy())
    except:
      print(artists, track_name)

K/DA;Madison Beer;(G)I-DLE;Lexie Liu;Jaira Burns;Seraphine;League of Legends MORE
K/DA;Madison Beer;(G)I-DLE;Jaira Burns;League of Legends POP/STARS
